## 1. 환경 설정

`(1) Env 환경변수`

In [38]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [9]:
import os
from glob import glob

## 2. LangChain RAG 구현

### 2.1 문서 로더 (Document Loaders)

In [10]:
# 텍스트 파일 목록 가져오기
txt_files = glob(os.path.join('data', '*_KR.txt'))

print(txt_files)

['data/리비안_KR.txt', 'data/테슬라_KR.txt']


In [11]:
# 첫 번째 텍스트 파일 내용 확인
from langchain_community.document_loaders import TextLoader

loader = TextLoader(txt_files[0], encoding='utf-8')
data = loader.load()
len(data)

1

In [12]:
# data type 확인
type(data)

list

In [13]:
# page_content를 출력
print(data[0].page_content)

리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다.

리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다. 리비안은 디젤 하이브리드 버전, 브라질 원메이크 시리즈를 위한 R1 GT 레이싱 버전, 4도어 세단 및 크로스오버 등 다양한 버전을 고려했습니다. 2011년에 프로토타입 해치백도 공개되었지만, R1과의 관계는 불명확합니다.

리비안은 2021년 10월 첫 번째 양산 차량인 R1T 트럭을 고객에게 인도하기 시작했습니다.



In [14]:
# metadata를 출력
print(data[0].metadata)

{'source': 'data/리비안_KR.txt'}


In [15]:
# Load all files from a directory
from tqdm import tqdm

data = []

for text_file in tqdm(txt_files):
    loader = TextLoader(text_file, encoding='utf-8')
    data += loader.load()

len(data)

100%|██████████| 2/2 [00:00<00:00, 1161.54it/s]


2

In [16]:
print(data[0].page_content)

리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다.

리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다. 리비안은 디젤 하이브리드 버전, 브라질 원메이크 시리즈를 위한 R1 GT 레이싱 버전, 4도어 세단 및 크로스오버 등 다양한 버전을 고려했습니다. 2011년에 프로토타입 해치백도 공개되었지만, R1과의 관계는 불명확합니다.

리비안은 2021년 10월 첫 번째 양산 차량인 R1T 트럭을 고객에게 인도하기 시작했습니다.



In [17]:
print(data[1].page_content)

테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. 머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.

2023년 테슬라는 1,808,581대의 차량을 판매하여 2022년에 비해 37.65% 증가했습니다. 2012년부터 2023년 3분기까지 테슬라의 전 세계 누적 판매량은 4,962,975대를 초과했습니다. SMT Packaging에 따르면, 2023년 테슬라의 판매량은 전 세계 전기차 시장의 약 12.9%를 차지했습니다.



In [18]:
type(data[0])

langchain_core.documents.base.Document

### 2.2 텍스트 분할기 (Text Splitters)

In [19]:
# 각 문서의 글자수를 계산
char_count = [len(doc.page_content) for doc in data]

print(char_count)

[514, 451]


In [20]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    chunk_size=250,        # 청크 크기  
    chunk_overlap=50,      # 청크 중 중복되는 부분 크기
    separator='\n\n',      # 청크를 나눌 때 사용할 구분자
)

texts = text_splitter.split_documents(data)
print(f"생성된 텍스트 청크 수: {len(texts)}")
print(f"각 청크의 길이: {list(len(text.page_content) for text in texts)}")

Created a chunk of size 282, which is longer than the specified 250
Created a chunk of size 268, which is longer than the specified 250


생성된 텍스트 청크 수: 5
각 청크의 길이: [175, 282, 52, 268, 180]


- CharacterTextSplitter는 단순히 문자 수로만 텍스트를 자르지 않습니다. 
- 기본적으로 문장이나 단락의 경계를 존중하려고 합니다. 
- 따라서 문장이나 단락의 끝부분에서 지정된 chunk_size를 약간 초과할 수 있습니다.

In [34]:
print("첫 번째 청크의 내용:", texts[0].page_content)

첫 번째 청크의 내용: 리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다.


In [35]:
print("두 번째 청크의 내용:", texts[1].page_content)

두 번째 청크의 내용: 리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했습니다. 이 차는 쉽게 교체 가능한 본체 패널을 갖춘 모듈식 캡슐 구조를 특징으로 하며, 2013년 말에서 2014년 초 사이에 생산이 예상되었습니다. 리비안은 디젤 하이브리드 버전, 브라질 원메이크 시리즈를 위한 R1 GT 레이싱 버전, 4도어 세단 및 크로스오버 등 다양한 버전을 고려했습니다. 2011년에 프로토타입 해치백도 공개되었지만, R1과의 관계는 불명확합니다.


In [36]:
# 겹치는 부분을 출력
print("첫 번째 청크의 최종 50글자:", texts[0].page_content[-50:])
print("두 번째 청크의 처음 50글자:", texts[1].page_content[:50])

첫 번째 청크의 최종 50글자: 했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다.
두 번째 청크의 처음 50글자: 리비안의 초기 모델은 스포츠카 R1(원래 이름은 Avera)로, 2+2 좌석의 미드 엔진 


In [5]:
text_splitter = CharacterTextSplitter(
    chunk_size=250,        # 청크 크기  
    chunk_overlap=50,      # 청크 중 중복되는 부분 크기
    separator='',          # 청크를 나눌 때 사용할 구분자
)

texts = text_splitter.split_documents(data)
print(f"생성된 텍스트 청크 수: {len(texts)}")
print(f"각 청크의 길이: {list(len(text.page_content) for text in texts)}")

NameError: name 'CharacterTextSplitter' is not defined

In [18]:
# 겹치는 부분을 출력
print("첫 번째 청크의 최종 50글자:", texts[0].page_content[-50:])
print("두 번째 청크의 처음 50글자:", texts[1].page_content[:50])

첫 번째 청크의 최종 50글자: 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했
두 번째 청크의 처음 50글자: 이름은 Avera)로, 2+2 좌석의 미드 엔진 하이브리드 쿠페로 피터 스티븐스가 디자인했


### 2.3 임베딩 모델 (Embeddings)

In [ ]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",  # 사용할 모델 이름을 지정 가능 
)
sample_text = "테슬라 창업자는 누구인가요?"
vector = embeddings.embed_query(sample_text)
print(f"임베딩 벡터의 차원: {len(vector)}")

In [21]:
import google.generativeai as genai
import os

# API 키 설정
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# 텍스트 임베딩 생성
sample_text = "테슬라 창업자는 누구인가요?"
result = genai.embed_content(
    model="models/text-embedding-004",  # 사용할 모델 이름
    content=sample_text                # 임베딩할 텍스트
)

# 임베딩 결과
vector = result['embedding']

# 결과 출력
print(f"임베딩 벡터의 차원: {len(vector)}")


임베딩 벡터의 차원: 768


### 2.4 벡터 저장소 (Vector Stores)

In [58]:
texts[0]

Document(metadata={'source': 'data/리비안_KR.txt'}, page_content='리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다.')

In [59]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(
    documents=texts, 
    embedding=embeddings,
    collection_name="chroma_test", 
    persist_directory="./chroma_db",
    )
print(f"벡터 저장소에 저장된 문서 수: {vectorstore._collection.count()}")

벡터 저장소에 저장된 문서 수: 5


In [22]:
from langchain_chroma import Chroma

class GeminiEmbeddings:
    def embed_documents(self, texts):
        """문서 리스트를 임베딩으로 변환"""
        return [
            genai.embed_content(
                model="models/text-embedding-004",
                content=text
            )['embedding']
            for text in texts
        ]

    def embed_query(self, text):
        """단일 텍스트 쿼리를 임베딩으로 변환"""
        result = genai.embed_content(
            model="models/text-embedding-004",
            content=text
        )
        return result['embedding']

# 커스텀 임베딩 클래스 초기화
embeddings = GeminiEmbeddings()

vectorstore = Chroma.from_documents(
    documents=texts, 
    embedding=embeddings,
    collection_name="chroma_test", 
    persist_directory="./chroma_db",
    )
print(f"벡터 저장소에 저장된 문서 수: {vectorstore._collection.count()}")

벡터 저장소에 저장된 문서 수: 10


In [24]:
# 문서 검색
query = "테슬라 창업자는 누구인가요?"
result = vectorstore.similarity_search(query)

print(f"검색 결과의 길이: {len(result)}")
print(f"검색 결과의 첫 번째 문서: {result[0].page_content[:100]}...")

검색 결과의 길이: 4
검색 결과의 첫 번째 문서: 리비안은 2021년 10월 첫 번째 양산 차량인 R1T 트럭을 고객에게 인도하기 시작했습니다....


### 2.5 검색기 (Retrievers)
   - 질의에 관련된 문서를 검색하는 컴포넌트

In [25]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

relevant_docs = retriever.invoke(query)
print(f"검색된 관련 문서 수: {len(relevant_docs)}")
print(f"첫 번째 관련 문서 내용 미리보기: {relevant_docs[0].page_content[:100]}...")

검색된 관련 문서 수: 2
첫 번째 관련 문서 내용 미리보기: 리비안은 2021년 10월 첫 번째 양산 차량인 R1T 트럭을 고객에게 인도하기 시작했습니다....


### 2.6 언어 모델 (LLMs)

In [39]:
# Context를 제공하지 않고 답변 생성 (Hallucination 발생 위험)

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",  # 사용할 모델 이름을 지정 가능
    temperature=0,        # 답변의 창의성을 조절
    max_tokens=100,       # 생성할 최대 토큰 수
    )

response = llm.invoke("테슬라 창업자는 누구인가요?")
print(f"LLM 응답: {response}")

PydanticUserError: The `__modify_schema__` method is not supported in Pydantic v2. Use `__get_pydantic_json_schema__` instead in class `SecretStr`.

For further information visit https://errors.pydantic.dev/2.10/u/custom-json-schema

In [42]:
# import google.generativeai as genai
# import os

# genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# from google.api_core import retry

# @retry.Retry()
# def generate_text(*args, **kwargs):
#   return genai.generate_text(*args, **kwargs)

# models = [m for m in genai.list_models() if 'generateText' in m.supported_generation_methods]
# model = models[0].name
# print(model)
from langchain_google_genai import ChatGoogleGenerativeAI
import os
# 다중 메시지 전송
# from langchain_core.prompts import ChatPromptTemplate

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.3,
    max_tokens=100,
    google_api_key=os.environ["GEMINI_API_KEY"]
)

response = llm.invoke("퍼플아이오 알아?")
print(f"LLM 응답: {response}")

LLM 응답: content=' 네, 퍼플아이오를 알고 있습니다. 퍼플아이오는 인공지능(AI) 기반의 이미지 생성 서비스입니다. 사용자가 원하는 이미지를 텍스트로 입력하면, AI가 그 이미지를 생성해줍니다. 퍼플아이오는 다양한 스타일과 해상도의 이미지를 생성할 수 있으며, 사용자는 생성된 이미지를 다운로드하거나 공유할 수 있습니다.\n\n퍼플아이' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'MAX_TOKENS', 'safety_ratings': []} id='run-daa327c6-3528-4c87-bc23-f7fbf3043b67-0' usage_metadata={'input_tokens': 8, 'output_tokens': 100, 'total_tokens': 108, 'input_token_details': {'cache_read': 0}}


In [ ]:
question = """
I have 77 houses, each with 31 cats.
Each cat owns 14 mittens, and 6 hats.
Each mitten was knit from 141m of yarn, each hat from 55m.
How much yarn was needed to make all the items?
"""

prompt_template = """
You are an expert at solving word problems. Here's one:

{question}

Work through it step by step, and show your work.
One step per line.

Your solution:
"""

completion = generate_text(
    model=model,
    prompt=prompt_template.format(question=question),
    # The maximum length of the response
    max_output_tokens=800,
)

final_prompt = prompt_template.format(question=question)
completion = generate_text(final_prompt)
print(completion)

print(completion.result)

NotFound: 404 Requested entity was not found.

In [62]:
import os
import google.generativeai as genai
from google.api_core import retry

genai.configure(api_key=os.environ["GEMINI_API_KEY"])
model = genai.GenerativeModel('gemini-1.5-flash')

@retry.Retry()
def generate_text(prompt):
    response = model.generate_content(prompt)
    return response.text

# Gemini AI Chat 클래스 정의
class ChatGeminiAI:
    def __init__(self, model, temperature=0, max_tokens=100):
        self.model = model = genai.GenerativeModel(model)
        self.temperature = temperature
        self.max_tokens = max_tokens
        genai.GenerationConfig(temperature=temperature, max_output_tokens=max_tokens)

    def invoke(self, prompt):
        response = generate_text(prompt)
        return response

#genai.GenerationConfig(temperature=0, max_output_tokens=100)
# LLM 객체 생성
llm = ChatGeminiAI(
    model='gemini-1.5-flash',  # 사용할 모델 이름 지정
    temperature=0,                 # 답변의 창의성을 조절
    max_tokens=100                 # 생성할 최대 토큰 수
)

# 질문에 대한 응답 생성
response = llm.invoke("테슬라 창업자는 누구인가요?")
print(f"LLM 응답: {response}")

LLM 응답: 테슬라 창업자는 일론 머스크입니다.  하지만 테슬라는 일론 머스크가 혼자서 만든 회사는 아니며, **Martin Eberhard**, **Marc Tarpenning**, **Ian Wright**, **JB Straubel**과 함께 2003년에 공동 창업했습니다.  일론 머스크는 초기 투자자로 참여했고, 나중에 회사의 CEO가 되었습니다.



In [21]:
import google.generativeai as genai
import os
from google.api_core import retry

# API 키 설정
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# 모델 설정 - text-only 모델 사용
model = genai.GenerativeModel('gemini-1.5-flash')
genai.GenerationConfig(temperature=0, max_output_tokens=100)

@retry.Retry()
def generate_text(prompt):
    response = model.generate_content(prompt)
    return response.text

# 문제 정의
question = """
I have 77 houses, each with 31 cats.
Each cat owns 14 mittens, and 6 hats.
Each mitten was knit from 141m of yarn, each hat from 55m.
How much yarn was needed to make all the items?
"""

# 프롬프트 템플릿
prompt_template = """
You are an expert at solving word problems. Here's one:

{question}

Work through it step by step, and show your work.
One step per line.

Your solution:
"""

# 텍스트 생성
final_prompt = prompt_template.format(question=question)
completion = generate_text(prompt=final_prompt)
print(completion)

Step 1: Calculate the total number of cats: 77 houses * 31 cats/house = 2387 cats

Step 2: Calculate the total number of mittens: 2387 cats * 14 mittens/cat = 33418 mittens

Step 3: Calculate the total meters of yarn for mittens: 33418 mittens * 141 m/mitten = 4713158 meters

Step 4: Calculate the total number of hats: 2387 cats * 6 hats/cat = 14322 hats

Step 5: Calculate the total meters of yarn for hats: 14322 hats * 55 m/hat = 787710 meters

Step 6: Calculate the total meters of yarn needed: 4713158 meters + 787710 meters = 5500868 meters

Therefore, a total of 5,500,868 meters of yarn was needed.



In [63]:
# Context를 제공하고 답변 생성

query_with_context = f"""{relevant_docs[0].page_content}\n\n위 내용에 근거하여 다음 질문에 답변하세요. \n\n{query}"""
print(query_with_context)

리비안은 2021년 10월 첫 번째 양산 차량인 R1T 트럭을 고객에게 인도하기 시작했습니다.

위 내용에 근거하여 다음 질문에 답변하세요. 

테슬라 창업자는 누구인가요?


In [65]:
response = llm.invoke(query_with_context)
print(f"LLM 응답: {response}")

LLM 응답: 질문에 대한 답은 제공된 텍스트에는 없습니다.  텍스트는 리비안의 양산 시작일을 언급하고 있을 뿐, 테슬라 창업자에 대한 정보는 포함하고 있지 않습니다.



### 2.7 전체 RAG 파이프라인 구성

In [36]:
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain



# LLM 초기화
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# 프롬프트 - 옵션 1 (허브 사용)
# https://smith.langchain.com/hub/langchain-ai/retrieval-qa-chat
# retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

# 프롬프트 - 옵션 2 (직접 작성)
retrieval_qa_chat_prompt = ChatPromptTemplate.from_template("""
다음 컨텍스트를 바탕으로 질문에 답변해주세요. 컨텍스트에 관련 정보가 없다면, 
"주어진 정보로는 답변할 수 없습니다."라고 말씀해 주세요.

컨텍스트: {context}

질문: {input}

답변:
""")


# 체인 생성
combine_docs_chain = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt)
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

# 체인 실행
query = "테슬라 창업자는 누구인가요?"
response = rag_chain.invoke({"input": query})

In [26]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# import google.generativeai as genai
import os

# genai.configure(api_key=os.environ["GEMINI_API_KEY"])

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",  # 사용할 모델 이름을 지정 가능
    temperature=0,                 # 답변의 창의성을 조절
    max_tokens=100,                 # 생성할 최대 토큰 수
    google_api_key=os.environ["GEMINI_API_KEY"]
)

retrieval_qa_chat_prompt = ChatPromptTemplate.from_template("""
다음 컨텍스트를 바탕으로 질문에 답변해주세요. 컨텍스트에 관련 정보가 없다면, 
"주어진 정보로는 답변할 수 없습니다."라고 말씀해 주세요.

컨텍스트: {context}

질문: {input}

답변:
""")


# 체인 생성
combine_docs_chain = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt)
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

# 체인 실행
query = "테슬라 창업자는 누구인가요?"
response = rag_chain.invoke({"input": query})

In [31]:
# 결과 출력
print(response)

{'input': '테슬라 창업자는 누구인가요?', 'context': [Document(metadata={'source': 'data/리비안_KR.txt'}, page_content='리비안은 2021년 10월 첫 번째 양산 차량인 R1T 트럭을 고객에게 인도하기 시작했습니다.'), Document(metadata={'source': 'data/리비안_KR.txt'}, page_content='리비안은 2021년 10월 첫 번째 양산 차량인 R1T 트럭을 고객에게 인도하기 시작했습니다.')], 'answer': '주어진 정보로는 답변할 수 없습니다.'}


In [30]:
# 응답 객체의 키 확인
response.keys()

dict_keys(['input', 'context', 'answer'])

In [32]:
response["answer"]

'주어진 정보로는 답변할 수 없습니다.'

`[추가 설명] create_stuff_documents_chain`
- 검색된 문서들을 하나의 컨텍스트로 결합하고, 이를 바탕으로 질문에 답변하는 체인을 생성

In [33]:

from langchain_core.documents import Document

stuff_chain = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt)

documents = [
    Document(page_content="LangChain은 LLM 애플리케이션 개발을 위한 프레임워크입니다."),
    Document(page_content="LangChain은 파이썬과 자바스크립트를 지원합니다.")
]
response = stuff_chain.invoke({
    "input": "LangChain은 어떤 언어를 지원하나요?",
    "context": documents
})
print(response)

LangChain은 파이썬과 자바스크립트를 지원합니다.


`[추가 설명] create_retrieval_chain`
- 질문에 관련된 문서를 검색하고, 검색된 문서를 바탕으로 답변을 생성하는 전체 RAG 파이프라인을 구축

In [34]:
# 검색 체인 생성
retrieval_chain = create_retrieval_chain(vectorstore.as_retriever(), stuff_chain)

# 체인 실행
query = "테슬라 창업자는 누구인가요?"
response = retrieval_chain.invoke({"input": query})

# 결과 출력
print(response)

{'input': '테슬라 창업자는 누구인가요?', 'context': [Document(metadata={'source': 'data/리비안_KR.txt'}, page_content='리비안은 2021년 10월 첫 번째 양산 차량인 R1T 트럭을 고객에게 인도하기 시작했습니다.'), Document(metadata={'source': 'data/리비안_KR.txt'}, page_content='리비안은 2021년 10월 첫 번째 양산 차량인 R1T 트럭을 고객에게 인도하기 시작했습니다.'), Document(metadata={'source': 'data/리비안_KR.txt'}, page_content='리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다.'), Document(metadata={'source': 'data/리비안_KR.txt'}, page_content='리비안은 MIT 박사 출신 RJ 스카린지가 2009년에 설립한 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율 전기차에 집중한 리비안은 2015년 대규모 투자를 통해 크게 성장하며 미시간과 베이 지역에 연구소를 설립했습니다. 주요 공급업체와의 접근성을 높이기 위해 본사를 미시간주 리보니아로 이전했습니다.')], 'answer': '주어진 정보로는 답변할 수 없습니다.'}


## 3. Gradio 챗봇

In [35]:
import gradio as gr

def answer_invoke(message, history):
    response = rag_chain.invoke({"input": message})
    return response["answer"]


# Graiio 인터페이스 생성 
demo = gr.ChatInterface(fn=answer_invoke, title="QA Bot")

# Graiio 실행  
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


/Users/jeongyeong-yeon/gy_jeon/rag-master-study/myenv/lib/python3.11/site-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


In [36]:
# Graiio 종료
demo.close()

Closing server running on port: 7860
